In [1]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
# import time
from pprint import pprint

# import googlemaps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.integrate import odeint

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)
from analysis.download_data import jh_git_url, population_data_url, export_foldname, all_files
from covid_forecast.utils import data_prep as dp
from covid_forecast.utils.data_prep import country_region_vname, s_0_vname, province_state_vname
from analysis.estimate_sir_params import estimate_sir_params, s_0_vname, population_df_vnames

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100
pd.options.display.precision = 8
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
export_dir = os.path.join(main_dir, "data", "raw"+os.sep)
countries_fname = "countries.csv"
print(main_dir+"\n"+export_dir)

D:\Users\gjnet\code_projects\covid_forecast
D:\Users\gjnet\code_projects\covid_forecast\data\raw\


# Open files as dataframes
Credit to dgrechka for locations_population. Data [source](https://www.kaggle.com/dgrechka/covid19-global-forecasting-locations-population/metadata)

In [3]:
population_data = pd.read_excel(population_data_url)
friendly_fnames = [file.replace("time_series_covid19_", "").replace(".csv", "") for file in all_files]
filename_dct = dict(zip(friendly_fnames, all_files))
pprint(filename_dct)

{'confirmed_global': 'time_series_covid19_confirmed_global.csv',
 'deaths_global': 'time_series_covid19_deaths_global.csv',
 'recovered_global': 'time_series_covid19_recovered_global.csv'}


In [4]:
df_dct = dict()
for file in filename_dct.keys():
    df_dct[file] = pd.read_csv(jh_git_url+filename_dct[file])  # reads csv from github repo
    print(file, df_dct[file].shape)
    display(df_dct[file].head(), df_dct[file].describe(include="all"), df_dct[file].dtypes)

confirmed_global (256, 74)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,NaN,Afghanistan,33.000,65.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174
1,NaN,Albania,41.153,20.168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243
2,NaN,Algeria,28.034,1.660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716
3,NaN,Andorra,42.506,1.522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376
4,NaN,Angola,-11.203,17.874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
count,79,256,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000,256.000
unique,79,180,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,Anhui,China,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,1,33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,21.824,23.159,2.168,2.555,3.676,5.602,8.273,11.434,21.789,24.086,32.164,38.777,47.023,65.574,77.660,93.328,107.949,120.289,134.340,145.000,156.836,167.039,175.008,176.645,235.812,261.270,269.648,278.219,286.164,293.500,295.465,297.645,300.074,306.922,308.430,310.785,314.086,317.922,323.227,328.562,335.980,345.191,352.758,362.656,371.562,382.367,397.660,413.465,428.988,443.711,463.359,491.699,501.375,567.207,609.770,654.117,709.273,769.930,839.145,947.266,"1,062.637","1,189.047","1,316.223","1,477.480","1,632.988","1,826.770","2,068.715","2,317.543","2,580.883","2,812.957","3,056.113","3,349.559"
std,NaN,NaN,24.484,71.121,27.806,27.905,34.738,48.344,67.557,91.021,223.404,224.759,309.736,367.217,451.884,702.301,848.997,"1,046.363","1,233.071","1,386.142","1,563.698","1,697.950","1,856.019","1,986.996","2,089.341","2,089.682","3,015.133","3,402.152","3,517.286","3,638.014","3,750.860","3,856.587","3,878.409","3,904.059","3,917.867","4,006.630","4,006.664","4,019.401","4,050.605","4,075.834","4,101.944","4,122.801","4,151.170","4,189.078","4,204.207","4,216.777","4,228.391","4,242.708","4,260.201","4,277.807","4,294.478","4,314.225","4,330.500","4,364.242","4,374.653","4,469.386","4,552.413","4,649.203","4,764.685","4,902.136","5,086.510","5,382.252","5,741.444","6,177.236","6,622.393","7,239.371","7,863.702","8,698.423","9,828.145","11,007.251","12,305.356","13,521.253","14,849.525","16,487.495"
min,NaN,NaN,-41.455,-135.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,NaN,NaN,7.845,-16.238,0.000,0.000,0.000,0.0

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

deaths_global (256, 74)


...

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

recovered_global (242, 74)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,NaN,Afghanistan,33.000,65.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,5
1,NaN,Albania,41.153,20.168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44,52
2,NaN,Algeria,28.034,1.660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,12,12,12,12,12,32,32,32,65,65,24,65,29,29,31,31,37,46
3,NaN,Andorra,42.506,1.522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,10,10
4,NaN,Angola,-11.203,17.874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
count,64,242,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000,242.000
unique,64,180,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
top,Anhui,China,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
freq,1,33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,20.543,29.173,0.116,0.124,0.149,0.161,0.215,0.252,0.442,0.521,0.591,0.917,1.174,1.950,2.574,3.521,4.645,6.145,8.310,10.810,13.405,16.306,19.351,21.281,26.012,33.298,38.822,44.897,51.996,59.306,66.616,75.112,78.058,94.570,96.669,104.244,115.310,125.554,137.508,151.698,164.388,176.512,188.438,199.289,211.446,222.298,230.847,241.149,250.802,258.240,266.132,276.872,282.331,290.293,300.099,314.190,322.678,334.050,344.264,351.136,361.240,378.893,404.541,406.409,446.281,470.194,504.752,540.971,576.095,616.041,680.025,735.678
std,NaN,NaN,24.142,67.721,1.800,1.804,1.999,2.066,2.709,2.916,5.173,5.691,5.852,9.166,10.970,19.156,25.092,33.976,41.415,53.520,72.987,94.371,117.602,145.355,172.514,176.888,226.571,310.740,365.545,430.976,509.775,591.073,669.060,762.569,769.598,988.487,991.942,"1,082.234","1,224.839","1,353.023","1,507.945","1,701.679","1,867.963","2,031.121","2,185.048","2,331.066","2,481.906","2,612.732","2,705.459","2,800.780","2,913.168","2,994.318","3,075.803","3,165.862","3,241.710","3,321.311","3,412.363","3,504.899","3,560.891","3,621.230","3,684.154","3,735.909","3,787.624","3,839.370","3,883.958","3,912.093","3,969.927","4,026.181","4,092.108","4,166.541","4,253.489","4,344.255","4,488.005","4,618.036"
min,NaN,NaN,-41.455,-106.347,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,NaN,NaN,7.135,-7.047,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
1/23/20             int64
1/24/20             int64
1/25/20             int64
1/26/20             int64
1/27/20             int64
1/28/20             int64
1/29/20             int64
1/30/20             int64
1/31/20             int64
2/1/20              int64
2/2/20              int64
2/3/20              int64
2/4/20              int64
2/5/20              int64
2/6/20              int64
2/7/20              int64
2/8/20              int64
2/9/20              int64
2/10/20             int64
2/11/20             int64
2/12/20             int64
2/13/20             int64
2/14/20             int64
2/15/20             int64
2/16/20             int64
2/17/20             int64
2/18/20             int64
2/19/20             int64
2/20/20             int64
2/21/20             int64
2/22/20             int64
2/23/20             int64
2/24/20             int64
2/25/20     

In [5]:
df_dct = dp.bulk_reduce_mem(df_dct)
# population_df = population_df.pipe(dp.reduce_mem)
gc.collect()

22

# Global measures dataframes

In [6]:
country_lvl_df_dct = dict()

for df_name in df_dct.keys():
    country_lvl_df_dct[df_name] = df_dct[df_name].drop(columns=[province_state_vname, "Lat", "Long"])
    country_lvl_df_dct[df_name] = country_lvl_df_dct[df_name].groupby(country_region_vname).sum().astype(int).reset_index(drop=False)
    country_lvl_df_dct[df_name][country_region_vname] = country_lvl_df_dct[df_name][country_region_vname].pipe(dp.normalise_str)

In [7]:
country_lvl_df_dct["confirmed_global"].head(2)

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20
0,AFGHANISTAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174
1,ALBANIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243


In [8]:
population_df = population_data.copy().rename(columns={"countriesAndTerritories": country_region_vname, "popData2018": s_0_vname})
population_df = population_df.sort_values("dateRep", ascending=False).reset_index(drop=True).drop_duplicates(country_region_vname)
population_df[country_region_vname] = population_df[country_region_vname].pipe(dp.normalise_str)
population_df[country_region_vname] = population_df[country_region_vname].replace(
    ["UNITED STATES OF AMERICA", "BRUNEI DARUSSALAM", "UNITED REPUBLIC OF TANZANIA", "CAPE VERDE", "DEMOCRATIC REPUBLIC OF THE CONGO",
     "CONGO", "COTE DIVOIRE", "CZECH REPUBLIC", "SOUTH KOREA"],
    ["US", "BRUNEI", "TANZANIA", "CABO VERDE", "CONGO (KINSHASA)",
     "CONGO (BRAZZAVILLE)", "COTE D'IVOIRE", "CZECHIA", "KOREA, SOUTH"], regex=False)

population_df = population_df.pipe(dp.fillna_or_add_val, "ERITREA", 5750433)
population_df = population_df.pipe(dp.fillna_or_add_val, "BURMA", 53708395)
population_df = population_df.pipe(dp.fillna_or_add_val, "DIAMOND PRINCESS", 3700)
population_df = population_df.pipe(dp.fillna_or_add_val, "MS ZAANDAM", 1243)
population_df = population_df.pipe(dp.fillna_or_add_val, "WEST BANK AND GAZA", 4685000)
population_df = population_df.pipe(dp.fillna_or_add_val, "ANGUILLA", 14731)
population_df = population_df.pipe(dp.fillna_or_add_val, "FALKLAND ISLANDS (MALVINAS)", 3398)

# population_df.head()

In [9]:
population_df[population_df[s_0_vname].isna()]

,dateRep,day,month,year,cases,deaths,Country/Region,geoId,countryterritoryCode,s_0


In [10]:
country_lvl_df_dct["confirmed_global"][country_region_vname][country_lvl_df_dct["confirmed_global"][country_region_vname].isna()]

Series([], Name: Country/Region, dtype: object)

In [11]:
# Fill in missing province/state
# # Start timer
# start_time = pd.to_datetime("now")
# time_fmt = "%d/%m/%Y %H:%M:%S"
# print("INFO: start time", start_time.strftime(time_fmt))

# # Filling missing province/state where possible
# gmaps = googlemaps.Client(key=os.getenv("gmaps_api_key"))

# fillna_keys = ["Province/State", "Country/Region"]
# loc_admin_long_vname = "long_name"

# # Start with the first df with missing province/state values
# temp_df = confirmed_global[match_keys][confirmed_global[prov_state_vname].isna()].copy()

# for i in temp_df.index:
#     row = temp_df.loc[i]

#     # Look up an address with reverse geocoding
#     try:
#         target_loc = gmaps.reverse_geocode((row["Lat"], row["Long"]))[0]['address_components']
#     except IndexError:
#         continue
#     admin_ind = dp.find_admin_loc(target_loc)
#     if admin_ind is not None:
#         temp_df.loc[i, prov_state_vname] = target_loc[dp.find_admin_loc(target_loc)][loc_admin_long_vname]
    
#     # Fill missing with country/region
#     temp_df[prov_state_vname] = temp_df[prov_state_vname].fillna(temp_df[country_region_vname])
    
#     # Gmaps anti-throttling tactic
#     time.sleep(1)  # wait 1 second before the next iteration

# # Fillna missing values
# for df in df_dct.keys():
#     df_dct[df][fillna_keys] = confirmed_global[fillna_keys].fillna(temp_df[fillna_keys])

# # Finish time
# finish_time = pd.to_datetime("now")
# elapsed_time_min = round((finish_time-start_time).total_seconds()/60, 2)
# print("INFO: finished. This took", elapsed_time_min, "minutes.")

# SIR
[Source code](https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/)

In [12]:
# # Total population, N.
# N = 1000

# # Initial number of infected and recovered individuals, I0 and R0.
# I0, R0 = 1, 0

# # Everyone else, S0, is susceptible to infection initially.
# S0 = N - I0 - R0

# # Contact rate (aka beta), and mean recovery rate aka (gamma) both (in 1/days).
# beta, gamma = 0.2, 1./10

# # A grid of time points (in days)
# days = 160
# t = np.linspace(0, days, days)

# # Initial conditions vector
# y0 = S0, I0, R0
# # Integrate the SIR equations over the time grid, t.
# ret = odeint(dp.deriv, y0, t, args=(N, beta, gamma))
# S, I, R = ret.T

# # Plot the data on three separate curves for S(t), I(t) and R(t)
# fig = plt.figure(facecolor='w')
# ax = fig.add_subplot(111, axisbelow=True)
# ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
# ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
# ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
# ax.set_facecolor('#dddddd')
# ax.set_xlabel('Time /days')
# ax.set_ylabel('Number (1000s)')
# ax.set_ylim(0,1.2)
# ax.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)
# ax.grid(b=True, which='major', c='w', lw=2, ls='-')
# legend = ax.legend()
# legend.get_frame().set_alpha(0.5)
# for spine in ('top', 'right', 'bottom', 'left'):
#     ax.spines[spine].set_visible(False)
# plt.show()

In [13]:
for df_name in country_lvl_df_dct.keys():
    country_lvl_df_dct[df_name] = country_lvl_df_dct[df_name].pipe(dp.safe_merge_cols, population_df[[country_region_vname, s_0_vname]])
    temp_var = country_lvl_df_dct[df_name][country_lvl_df_dct[df_name][s_0_vname].isna()]
    print("WARNING: the following countries have no population values and are excluded", ", ".join(list(temp_var[country_region_vname])))
    country_lvl_df_dct[df_name] = country_lvl_df_dct[df_name].drop(index=temp_var.index)
    country_lvl_df_dct[df_name] = country_lvl_df_dct[df_name].sort_values(s_0_vname)
    exec(df_name+" = country_lvl_df_dct['"+df_name+"']")  # creates a reference for friendly_names

country_lvl_df_dct = dp.bulk_reduce_mem(country_lvl_df_dct)
gc.collect()

INFO: successfully merged
INFO: successfully merged
INFO: successfully merged


0

## Estimating SIR beta and gamma parameters
[Credit Lewuathe - source](https://github.com/Lewuathe/COVID19-SIR)

In [14]:
countries = confirmed_global[confirmed_global[country_region_vname].str.contains(
    r"cyprus|italy|spain|united kingdom|^us$|india|china", case=False)].drop_duplicates()
display(countries[population_df_vnames].head())

,Country/Region,s_0
44,CYPRUS,1189265
153,SPAIN,46723749
84,ITALY,60431283
172,UNITED KINGDOM,66488991
168,US,327167434


In [ ]:
%%time
results_df = estimate_sir_params(countries=countries[country_region_vname], df_dct=country_lvl_df_dct, predict_range=365, plot=True)

CYPRUS population: 1189265 start date 3/9/20
country=CYPRUS, beta=0.00000020, gamma=0.02077768, r_0=0.00000970 

SPAIN population: 46723749 start date 2/1/20
country=SPAIN, beta=0.00000001, gamma=0.32810844, r_0=0.00000003 

ITALY population: 60431283 start date 1/31/20
country=ITALY, beta=0.00000001, gamma=0.40000000, r_0=0.00000002 

UNITED KINGDOM population: 66488991 start date 1/31/20
country=UNITED KINGDOM, beta=0.00000001, gamma=0.40000000, r_0=0.00000002 

US population: 327167434 start date 1/22/20
country=US, beta=0.00000001, gamma=0.00000005, r_0=0.18688683 

INDIA population: 1352617328 start date 1/30/20


## Export results

In [ ]:
results_df.to_csv(export_foldname+"sir_params_per_country.csv")

In [ ]:
# TODO: increase estimation speed or use other SIR methods: https://www.kaggle.com/lisphilar/covid-19-data-with-sir-model
# TODO: (future) return date or date_range with beta and gamma

# Resources:
# TODO: investigate ICL report and code https://www.imperial.ac.uk/news/196234/covid19-imperial-researchers-model-likely-impact/
# https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-Europe-estimates-and-NPI-impact-30-03-2020.pdf
# https://github.com/ImperialCollegeLondon/covid19model/releases/tag/v1.0